In [1]:
import os
import numpy as np
from torch.utils.data import Dataset

In [3]:
class TrialDataset(Dataset):
    #Characterizes a dataset for PyTorch'
    def __init__(self, feature_path, label_path):
        'Initialization'
        self.features = np.load(feature_path)
        self.labels = np.load(label_path)

    def __len__(self):
        'Denotes the total number of samples'
        return np.shape(self.labels)[0]

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        #ID = self.list_IDs[index]

        # Load data and get label
        #X = torch.load('data/' + ID + '.pt')
        
        #y = self.labels[ID]

        return self.features[index], self.labels[index]

In [ ]:
class TrialDataset(Dataset):
    #Characterizes a dataset for PyTorch'
    def __init__(self, feature_path, label_path, list_name):
        'Initialization'
        self.features = np.load(feature_path)
        self.labels = np.load(label_path)

    def __len__(self):
        'Denotes the total number of samples'
        return np.shape(self.labels)[0]

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        #ID = self.list_IDs[index]

        # Load data and get label
        #X = torch.load('data/' + ID + '.pt')
        
        #y = self.labels[ID]

        return self.features[index], self.labels[index]

In [4]:
training_set = TrialDataset("/nfs/amino-home/qingyliu/dihedral_angle/ML_data/back_tag/back_train_feature_new_0.npy","/nfs/amino-home/qingyliu/dihedral_angle/ML_data/back_tag/back_train_label_new_0.npy")

In [9]:
import torch

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True

In [11]:
# Parameters
params = {'batch_size': 2,
          'shuffle': True,
          'num_workers': 1}
max_epochs = 100

In [15]:
training_generator = torch.utils.data.DataLoader(training_set, **params)

In [26]:
fea,lab = next(iter(training_generator))

In [29]:
lab.shape

torch.Size([2])

In [19]:
class Net(torch.nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = torch.nn.Sequential(
            # Defining a 2D convolution layer
            torch.nn.Conv1d(27,16 , kernel_size=3, stride=2),
            torch.nn.BatchNorm1d(16),
            torch.nn.ReLU(inplace=True),
            # Defining another 2D convolution layer
            torch.nn.Conv1d(16, 8, kernel_size=3, stride=2),
            torch.nn.BatchNorm1d(8),
            torch.nn.ReLU(inplace=True),
            # Defining another 2D convolution layer
            torch.nn.Conv1d(8, 3, kernel_size=3),
            torch.nn.BatchNorm1d(3),
            
        )

        self.linear_layers = torch.nn.Sequential(
            torch.nn.Softmax()
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.linear_layers(x)
        return x

In [34]:
# defining the model
model = Net()
# defining the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = torch.nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv1d(27, 16, kernel_size=(3,), stride=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(16, 8, kernel_size=(3,), stride=(2,))
    (4): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv1d(8, 3, kernel_size=(3,), stride=(1,))
    (7): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear_layers): Sequential(
    (0): Softmax(dim=None)
  )
)


def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [47]:
def train(epoch):
    model.train()    # Training
    for idx, (local_batch, local_labels) in enumerate(training_generator):
        # Transfer to GPU
        local_batch, local_labels = np.swapaxes(local_batch,2,1).to(device), local_labels.to(device)
        optimizer.zero_grad()
        output_train = model(local_batch.float())
        loss_train = criterion(output_train, local_labels.unsqueeze(1))
        loss_train.backward()
        optimizer.step()
        tr_loss = loss_train.item()
        if idx%20 == 0:
            print('Epoch : ',epoch+1, '\t', 'loss :', tr_loss)

In [37]:
local_batch, local_labels = next(iter(training_generator))
local_batch, local_labels = np.swapaxes(local_batch,2,1).to(device), local_labels.to(device)
optimizer.zero_grad()
output_train = model(local_batch.float())

In [39]:
output_train.shape

torch.Size([2, 3, 1])

In [43]:
loss_train = criterion(output_train, local_labels.unsqueeze(1))

In [44]:
loss_train

tensor(1.1811, grad_fn=<NllLoss2DBackward>)

In [48]:
max_epochs = 2
# Loop over epochs
for epoch in range(max_epochs):
    train(epoch)

        


Epoch :  1 	 loss : 0.7800476551055908
Epoch :  1 	 loss : 0.9088211059570312
Epoch :  1 	 loss : 1.1220486164093018
Epoch :  1 	 loss : 1.2593350410461426
Epoch :  1 	 loss : 1.0075796842575073
Epoch :  1 	 loss : 1.1046528816223145
Epoch :  1 	 loss : 1.1087853908538818
Epoch :  1 	 loss : 1.3373603820800781
Epoch :  1 	 loss : 1.1054155826568604
Epoch :  1 	 loss : 1.1165270805358887
Epoch :  1 	 loss : 1.0998907089233398
Epoch :  1 	 loss : 1.104768991470337
Epoch :  1 	 loss : 1.1115697622299194
Epoch :  1 	 loss : 1.1150426864624023
Epoch :  1 	 loss : 1.1962180137634277
Epoch :  1 	 loss : 1.4340628385543823
Epoch :  1 	 loss : 1.1315393447875977
Epoch :  1 	 loss : 1.0763921737670898
Epoch :  1 	 loss : 1.1982696056365967
Epoch :  1 	 loss : 0.9020311236381531
Epoch :  1 	 loss : 1.1054577827453613
Epoch :  1 	 loss : 1.1780321598052979
Epoch :  1 	 loss : 0.8247377872467041
Epoch :  1 	 loss : 1.0994203090667725
Epoch :  1 	 loss : 1.4049897193908691
Epoch :  1 	 loss : 1.1084

Epoch :  1 	 loss : 0.9113308787345886
Epoch :  1 	 loss : 1.1072015762329102
Epoch :  1 	 loss : 1.1052911281585693
Epoch :  1 	 loss : 1.2233737707138062
Epoch :  1 	 loss : 1.1026105880737305
Epoch :  1 	 loss : 1.1003574132919312
Epoch :  1 	 loss : 1.0668964385986328
Epoch :  1 	 loss : 1.0354061126708984
Epoch :  1 	 loss : 1.205305814743042
Epoch :  1 	 loss : 1.1569230556488037
Epoch :  1 	 loss : 1.101179599761963
Epoch :  1 	 loss : 1.1239385604858398
Epoch :  1 	 loss : 0.8403978943824768
Epoch :  1 	 loss : 1.1328952312469482
Epoch :  1 	 loss : 0.8359388113021851
Epoch :  1 	 loss : 1.2671775817871094
Epoch :  1 	 loss : 0.9201783537864685
Epoch :  1 	 loss : 1.1190242767333984
Epoch :  1 	 loss : 1.1764509677886963
Epoch :  1 	 loss : 1.0476497411727905
Epoch :  1 	 loss : 1.220814824104309
Epoch :  1 	 loss : 1.1197988986968994
Epoch :  1 	 loss : 0.9747066497802734
Epoch :  1 	 loss : 1.1032776832580566
Epoch :  1 	 loss : 1.0986509323120117
Epoch :  1 	 loss : 1.127003

Epoch :  1 	 loss : 1.099870204925537
Epoch :  1 	 loss : 1.0955932140350342
Epoch :  1 	 loss : 0.955715537071228
Epoch :  1 	 loss : 1.103037714958191
Epoch :  1 	 loss : 1.101621389389038
Epoch :  1 	 loss : 1.0196294784545898
Epoch :  1 	 loss : 1.1640504598617554
Epoch :  1 	 loss : 1.01503324508667
Epoch :  1 	 loss : 1.1115634441375732
Epoch :  1 	 loss : 0.9788406491279602
Epoch :  1 	 loss : 1.103403091430664
Epoch :  1 	 loss : 0.9551838636398315
Epoch :  1 	 loss : 1.1058063507080078
Epoch :  1 	 loss : 1.0759696960449219
Epoch :  1 	 loss : 1.1068267822265625
Epoch :  1 	 loss : 1.1035585403442383
Epoch :  1 	 loss : 1.1069796085357666
Epoch :  1 	 loss : 1.0548748970031738
Epoch :  1 	 loss : 1.1722395420074463
Epoch :  1 	 loss : 1.103191614151001
Epoch :  1 	 loss : 1.1006872653961182
Epoch :  1 	 loss : 1.1144897937774658
Epoch :  1 	 loss : 1.1172595024108887
Epoch :  1 	 loss : 1.2204854488372803
Epoch :  1 	 loss : 1.1729331016540527
Epoch :  1 	 loss : 1.08615243434

Epoch :  1 	 loss : 1.1016066074371338
Epoch :  1 	 loss : 1.1919021606445312
Epoch :  1 	 loss : 1.160236120223999
Epoch :  1 	 loss : 1.0993984937667847
Epoch :  1 	 loss : 1.0772042274475098
Epoch :  1 	 loss : 1.106905221939087
Epoch :  1 	 loss : 0.9845846891403198
Epoch :  1 	 loss : 1.1024938821792603
Epoch :  1 	 loss : 1.0996756553649902
Epoch :  1 	 loss : 1.091254711151123
Epoch :  1 	 loss : 1.1823394298553467
Epoch :  1 	 loss : 1.1014859676361084
Epoch :  1 	 loss : 1.0595049858093262
Epoch :  1 	 loss : 1.185384750366211
Epoch :  1 	 loss : 1.10347318649292
Epoch :  1 	 loss : 1.1145296096801758
Epoch :  1 	 loss : 1.0178521871566772
Epoch :  1 	 loss : 1.0543067455291748
Epoch :  1 	 loss : 1.1393179893493652
Epoch :  1 	 loss : 1.1031676530838013
Epoch :  1 	 loss : 1.1047608852386475
Epoch :  1 	 loss : 1.1193406581878662
Epoch :  1 	 loss : 1.1132512092590332
Epoch :  1 	 loss : 1.1903742551803589
Epoch :  1 	 loss : 1.0214232206344604
Epoch :  1 	 loss : 1.071420192

Epoch :  1 	 loss : 1.106654167175293
Epoch :  1 	 loss : 1.1078252792358398
Epoch :  1 	 loss : 1.102004885673523
Epoch :  1 	 loss : 1.0973029136657715
Epoch :  1 	 loss : 1.2331478595733643
Epoch :  1 	 loss : 1.0533232688903809
Epoch :  1 	 loss : 1.1179901361465454
Epoch :  1 	 loss : 1.046383261680603
Epoch :  1 	 loss : 1.1718428134918213
Epoch :  1 	 loss : 1.335388422012329
Epoch :  1 	 loss : 1.1025242805480957
Epoch :  1 	 loss : 0.9961421489715576
Epoch :  1 	 loss : 1.0257301330566406
Epoch :  1 	 loss : 0.9828979969024658
Epoch :  1 	 loss : 0.9524497389793396
Epoch :  1 	 loss : 0.9998348951339722
Epoch :  1 	 loss : 1.1054604053497314
Epoch :  1 	 loss : 1.1008830070495605
Epoch :  1 	 loss : 1.053968071937561
Epoch :  1 	 loss : 1.1024713516235352
Epoch :  1 	 loss : 1.0129952430725098
Epoch :  1 	 loss : 1.1097910404205322
Epoch :  1 	 loss : 1.1989713907241821
Epoch :  1 	 loss : 0.964819073677063
Epoch :  1 	 loss : 1.1015675067901611
Epoch :  1 	 loss : 1.032470226

...

Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.098

Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.098

Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.098

Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.0986123085021973
Epoch :  1 	 loss : 1.098

KeyboardInterrupt: 

In [5]:




# Validation
with torch.set_grad_enabled(False):
    for local_batch, local_labels in validation_generator:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)


(array([[0.2350522 , 0.02231344, 0.80533842, 0.98579704, 0.02847059,
         0.09449037, 0.4061269 , 0.0319844 , 0.88894403, 0.04787969,
         0.13124447, 0.41580948, 0.18242552, 0.9393461 , 0.84553473,
         0.39651675, 0.25161828, 0.05896701, 0.04228977, 0.09708864,
         0.999     , 0.001     , 0.        , 0.09701467, 0.33574106,
         0.56724427, 0.        ],
        [0.65475346, 0.9793667 , 0.21585281, 0.28090034, 0.06180347,
         0.21081829, 0.97234758, 0.06238585, 0.26114999, 0.06122616,
         0.12238889, 0.4061269 , 0.9393461 , 0.31647911, 0.18542742,
         0.95689275, 0.53245431, 0.11105597, 0.03992533, 0.21081829,
         0.856     , 0.123     , 0.01      , 0.33333333, 0.33333333,
         0.33333333, 0.        ],
        [0.6637387 , 0.82635335, 0.10340045, 0.30576366, 0.03992533,
         0.09708864, 0.68997448, 0.04436183, 0.75212911, 0.07242649,
         0.13705129, 0.24048908, 0.74838172, 0.53245431, 0.99517928,
         0.31002552, 0.2042403 , 0.

In [ ]:
class StreamDataset(Dataset):
    #Characterizes a dataset for PyTorch'
    def __init__(self, feature_path, label_path):
        'Initialization'
        self.feature = labels
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.load('data/' + ID + '.pt')
        y = self.labels[ID]

        return X, y
    